In [1]:
import math # for log

In [2]:
DISTINCT = 27
MY_ID = 1
CHECK = ord('-')-ord('a')
FIX = 26 # for CHECK

In [3]:
def clean(msg):
    msg = msg.lower()
    clean_msg = ''
    
    for ch in msg:
        if 'a' <= ch <= 'z':
            clean_msg += ch
    return clean_msg

In [4]:
import random
def generate_key_vignere(length=20, distinct=True):
    """
        key length should be <= 26, else distinct not possible.
    """
    if distinct and length<=26:
        all_idx = [i for i in range(26)]
        random.shuffle(all_idx)
        return ''.join([chr(idx + ord('a')) for idx in all_idx[:length]])
    else:
        all_idx = [random.randint(0,25) for i in range(length)]
        return ''.join([chr(idx + ord('a')) for idx in all_idx])

In [5]:
# num_to_english = {1:'one', 2:'two', 3:'three', 4:'four', 5:'five', 6:'six', 7:'seven', 8:'eight', 9:'nine', 10:'ten', 11:'eleven', 12:'twelve', 13:'thirteen', 14:'fourteen', 15:'fifteen', 16:'sixteen', 17:'seventeen', 18:'eighteen', 19:'nineteen', 20:'twenty', 21:'twentyone', 22:'twentytwo', 23:'twentythree', 24:'twentyfour', 25:'twentyfive', 26:'twentysix'}

# key_len = 20
# key = generate_key_vignere(key_len)

# msg = 'hellothis is a beautiful day!'
# msg = clean(msg)
# msg = num_to_english[key_len] + key + msg
# msg

In [6]:
key_vig = generate_key_vignere()

msg = 'hellothis is a beautiful day!'
msg = clean(msg)
msg, key_vig

('hellothisisabeautifulday', 'hykfmxroeadnbgzpcviw')

In [7]:
def encrypt_vignere(pl_text, key):
    """
        Takes plain_text(Lowercase Letters) and key as argument and returns encrypted cipher_text.
    """
    ci_text = ''
    key_len = len(key)
    for i in range(len(pl_text)):
        ci_char = ( ord(pl_text[i]) - ord('a') + ord(key[i%key_len]) - ord('a') )%26
        ci_text += chr( ci_char + ord('a') )
    
    return ci_text

In [8]:
ci_text_vig = encrypt_vignere(msg, key_vig)

msg_for_RSA = key_vig + '-' + ci_text_vig
msg_for_RSA

'hykfmxroeadnbgzpcviw-ocvqaqywwivnckzjvdnqsbkd'

In [9]:
cat private_1.txt

65537 18257179258442105089 5915587277 5915587277


In [13]:
def find_block_size(num_diff_chars, n):
    block_size = int(math.log(n, num_diff_chars))

    if num_diff_chars**block_size == n:
        return block_size-1
    else:
        return block_size

In [31]:
def mod_pow(x, n, m):
    ans = 1
    for i in range(n):
        ans = ans*x % m
    return ans

In [90]:
def encrypt_RSA(pl_text, key, n):
    ci_text = ''
    r = find_block_size(DISTINCT, n)
    s = r+1
    for i in range(0, len(pl_text), r):
        block = pl_text[i:i+r]
        M = 0
        for j, ch in enumerate(block):
            if ch=='-':
                M += (FIX)*(DISTINCT**j)
            else:
                M += (ord(ch) - ord('a'))*(DISTINCT**j)
#         C = M**key % n
        C = mod_pow(M, key, n)
        for j in range(s-1,-1,-1):
            temp = C // (DISTINCT**j)
            C -= temp*(DISTINCT**j)
            if temp == 26: temp = CHECK # for '-'
            
            ci_text += chr(temp+ord('a'))
        
    return ci_text

In [96]:
e = 7
p = 149
q = 151
phi_n = (p-1)*(q-1)

d=0
for i in range(2, phi_n):
    if i*e % phi_n == 1:
        d=i%phi_n
        break
d

6343

In [97]:
# e, d, p, q = map(int, open('private_' + str(MY_ID) + '.txt', 'r').read().split())

encrypt_RSA(msg_for_RSA, e, p*q)

'aasvacdealppabala-oiamgiavsybahsamzeajwiaxubatiwaavwallfacox'

In [98]:
def decrypt_RSA(ci_text, key, n):
    pl_text = ''
    r = find_block_size(DISTINCT, n)
    s = r+1
    
    for i in range(0, len(ci_text), s):
        block = ci_text[i:i+s]
        C = 0
        for j, ch in enumerate(block[::-1]): # Reverse of encryption
            if ch=='-':
                C += (FIX)*(DISTINCT**j)
            else:
                C += (ord(ch) - ord('a'))*(DISTINCT**j)
#         C = M**key % n
        M = mod_pow(C, key, n)
    
        updated_block = ''
        for j in range(r-1,-1,-1):
            temp = M // (DISTINCT**j)
            M -= temp*(DISTINCT**j)
            if temp == 26: temp = CHECK # for '-'
            
            updated_block += chr(temp+ord('a'))
        pl_text += updated_block[::-1]
    
    return pl_text

In [99]:
decrypt_RSA('aasvacdealppabala-oiamgiavsybahsamzeajwiaxubatiwaavwallfacox',d,p*q)

'hykfmxroeadnbgzpcviw-ocvqaqywwivnckzjvdnqsbkd'